# 06 - Modeling: Battle Outcome Prediction

**Purpose**: Build predictive models for technical rigor scoring.

**Goal**: Predict battle outcomes based on deck composition alone.

**Models to Try**:
1. Logistic Regression (baseline)
2. Random Forest (feature importance insights)
3. XGBoost (likely best performance)

**Key Metrics**:
- Accuracy
- Precision/Recall
- ROC-AUC
- Feature importance (for insights!)

In [1]:
import sys, os, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import xgboost as xgb

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))

from visualization import setup_presentation_style
setup_presentation_style()

✓ Presentation style configured


## 1. Load Feature Matrix

In [ ]:
# Try to load engineered features from notebook 05
# If not available, create a basic feature set
try:
    features = pd.read_parquet(os.path.join(PROJECT_ROOT, 'artifacts/model_features.parquet'))
    print(f"✓ Loaded {len(features):,} battles with {len(features.columns)} features")
except FileNotFoundError:
    print("⚠ Feature matrix not found. Creating basic feature set from raw data...")
    # Create a sample directly from battles view
    import sys
    sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))
    from duckdb_utils import get_connection, create_battles_view, query_to_df
    
    con = get_connection()
    create_battles_view(con, os.path.join(PROJECT_ROOT, 'battles.parquet'))
    
    sample_query = """
    SELECT * FROM battles
    USING SAMPLE 10% (bernoulli)
    """
    features = query_to_df(con, sample_query, show_progress=False)
    print(f"✓ Created sample with {len(features):,} battles")

print(f"\nDataset shape: {features.shape}")
print(f"Columns: {list(features.columns[:10])}...")  # Show first 10 columns

## 2. Prepare Data for Modeling

In [ ]:
# Restructure data: each battle becomes 2 rows (one for each player)
# Row 1: winner's perspective (outcome=1)
# Row 2: loser's perspective (outcome=0)

print("Restructuring data for binary classification...")

# Select features to use for modeling
feature_cols = []

# Add numerical features
numerical_features = [
    'winner.elixir.average', 'winner.troop.count', 'winner.spell.count', 
    'winner.structure.count', 'winner.rarity.legendary.count',
    'winner.rarity.epic.count', 'winner.rarity.rare.count', 'winner.rarity.common.count',
    'winner.startingTrophies'
]

# Check which columns exist
for col in numerical_features:
    if col in features.columns:
        feature_cols.append(col)

print(f"Using {len(feature_cols)} features for modeling")

# Create winner rows (outcome=1)
winner_data = features[feature_cols].copy()
winner_data['outcome'] = 1

# Create loser rows (outcome=0) by replacing 'winner' with 'loser' in column names
loser_feature_cols = [col.replace('winner', 'loser') for col in feature_cols]
loser_data = features[loser_feature_cols].copy()
loser_data.columns = feature_cols  # Rename to match winner columns
loser_data['outcome'] = 0

# Combine winner and loser rows
modeling_data = pd.concat([winner_data, loser_data], ignore_index=True)

# Drop rows with missing values
modeling_data = modeling_data.dropna()

print(f"\n✓ Restructured dataset:")
print(f"  - Original battles: {len(features):,}")
print(f"  - Total rows (2x battles): {len(modeling_data):,}")
print(f"  - Features: {len(feature_cols)}")
print(f"  - Outcome distribution:")
print(f"    • Wins: {(modeling_data['outcome'] == 1).sum():,} ({(modeling_data['outcome'] == 1).mean()*100:.1f}%)")
print(f"    • Losses: {(modeling_data['outcome'] == 0).sum():,} ({(modeling_data['outcome'] == 0).mean()*100:.1f}%)")

# Define X and y
y = modeling_data['outcome']
X = modeling_data.drop('outcome', axis=1)

print(f"\n✓ Ready for model training")
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")

## 3. Train/Test Split

In [ ]:
# Create train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✓ Train/Test Split Complete:")
print(f"  Training set: {len(X_train):,} samples ({len(X_train) / len(X) * 100:.1f}%)")
print(f"  Test set: {len(X_test):,} samples ({len(X_test) / len(X) * 100:.1f}%)")
print(f"\n  Training outcome distribution:")
print(f"    • Wins: {(y_train == 1).sum():,} ({(y_train == 1).mean() * 100:.1f}%)")
print(f"    • Losses: {(y_train == 0).sum():,} ({(y_train == 0).mean() * 100:.1f}%)")
print(f"\n  Test outcome distribution:")
print(f"    • Wins: {(y_test == 1).sum():,} ({(y_test == 1).mean() * 100:.1f}%)")
print(f"    • Losses: {(y_test == 0).sum():,} ({(y_test == 0).mean() * 100:.1f}%)")

## 4. Model 1: Logistic Regression (Baseline)

In [ ]:
# Train Logistic Regression (baseline model)
print("Training Logistic Regression...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr_model.fit(X_train, y_train)

# Predictions
lr_pred = lr_model.predict(X_test)
lr_pred_proba = lr_model.predict_proba(X_test)[:, 1]

# Evaluation metrics
lr_acc = accuracy_score(y_test, lr_pred)
lr_roc_auc = roc_auc_score(y_test, lr_pred_proba)

print(f"\n✓ Logistic Regression Results:")
print(f"  Accuracy: {lr_acc:.4f} ({lr_acc*100:.2f}%)")
print(f"  ROC-AUC: {lr_roc_auc:.4f}")
print(f"\n  Classification Report:")
print(classification_report(y_test, lr_pred, target_names=['Loss', 'Win']))

# Confusion matrix
cm = confusion_matrix(y_test, lr_pred)
print(f"\n  Confusion Matrix:")
print(f"    True Negatives: {cm[0,0]:,}  |  False Positives: {cm[0,1]:,}")
print(f"    False Negatives: {cm[1,0]:,} |  True Positives: {cm[1,1]:,}")

## 5. Model 2: Random Forest

In [ ]:
# Train Random Forest
print("Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10)
rf_model.fit(X_train, y_train)

# Predictions
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Evaluation metrics
rf_acc = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)

print(f"\n✓ Random Forest Results:")
print(f"  Accuracy: {rf_acc:.4f} ({rf_acc*100:.2f}%)")
print(f"  ROC-AUC: {rf_roc_auc:.4f}")
print(f"\n  Classification Report:")
print(classification_report(y_test, rf_pred, target_names=['Loss', 'Win']))

# Confusion matrix
cm = confusion_matrix(y_test, rf_pred)
print(f"\n  Confusion Matrix:")
print(f"    True Negatives: {cm[0,0]:,}  |  False Positives: {cm[0,1]:,}")
print(f"    False Negatives: {cm[1,0]:,} |  True Positives: {cm[1,1]:,}")

# Store feature importances for later analysis
rf_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

## 6. Model 3: XGBoost

In [ ]:
# Train XGBoost
print("Training XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=6,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

# Predictions
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Evaluation metrics
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_roc_auc = roc_auc_score(y_test, xgb_pred_proba)

print(f"\n✓ XGBoost Results:")
print(f"  Accuracy: {xgb_acc:.4f} ({xgb_acc*100:.2f}%)")
print(f"  ROC-AUC: {xgb_roc_auc:.4f}")
print(f"\n  Classification Report:")
print(classification_report(y_test, xgb_pred, target_names=['Loss', 'Win']))

# Confusion matrix
cm = confusion_matrix(y_test, xgb_pred)
print(f"\n  Confusion Matrix:")
print(f"    True Negatives: {cm[0,0]:,}  |  False Positives: {cm[0,1]:,}")
print(f"    False Negatives: {cm[1,0]:,} |  True Positives: {cm[1,1]:,}")

# Store feature importances for later analysis
xgb_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

## 7. Feature Importance Analysis



In [ ]:
# Visualize feature importance from best model (use Random Forest)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest feature importance
top_rf = rf_feature_importance.head(15)
ax1.barh(top_rf['feature'], top_rf['importance'], color='steelblue', edgecolor='black', alpha=0.8)
ax1.set_xlabel('Importance Score', fontsize=14)
ax1.set_title('Top 15 Features: Random Forest', fontsize=16, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(axis='x', alpha=0.3)

# XGBoost feature importance
top_xgb = xgb_feature_importance.head(15)
ax2.barh(top_xgb['feature'], top_xgb['importance'], color='coral', edgecolor='black', alpha=0.8)
ax2.set_xlabel('Importance Score', fontsize=14)
ax2.set_title('Top 15 Features: XGBoost', fontsize=16, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Top 10 Most Important Features (Random Forest):")
print(rf_feature_importance.head(10).to_string(index=False))

print("\n💡 Key Insights:")
print(f"  - Most important feature: {rf_feature_importance.iloc[0]['feature']}")
print(f"  - Top 3 features account for {rf_feature_importance.head(3)['importance'].sum()*100:.1f}% of total importance")
print(f"  - These features tell us what matters most for winning battles!")

## 8. Model Evaluation Summary

In [ ]:
# Create comprehensive model comparison
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [lr_acc, rf_acc, xgb_acc],
    'ROC-AUC': [lr_roc_auc, rf_roc_auc, xgb_roc_auc]
})

# Add percentage formatting
results['Accuracy (%)'] = results['Accuracy'] * 100
results['ROC-AUC (%)'] = results['ROC-AUC'] * 100

print("=" * 70)
print("📊 MODEL PERFORMANCE SUMMARY")
print("=" * 70)
print(results[['Model', 'Accuracy (%)', 'ROC-AUC (%)']].to_string(index=False))
print("=" * 70)

# Identify best model
best_model_idx = results['Accuracy'].idxmax()
best_model_name = results.loc[best_model_idx, 'Model']
best_acc = results.loc[best_model_idx, 'Accuracy (%)']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Accuracy: {best_acc:.2f}%")
print(f"   ROC-AUC: {results.loc[best_model_idx, 'ROC-AUC (%)']:.2f}%")

# Compare to benchmark
benchmark_acc = 56.94
print(f"\n📈 Comparison to Research Benchmark:")
print(f"   Previous research: {benchmark_acc}%")
print(f"   Our best model: {best_acc:.2f}%")
if best_acc > benchmark_acc:
    print(f"   ✅ We beat the benchmark by {best_acc - benchmark_acc:.2f} percentage points!")
else:
    print(f"   ⚠ We're {benchmark_acc - best_acc:.2f} percentage points below the benchmark")

print(f"\n💡 Key Takeaways for Presentation:")
print(f"   1. Achieved {best_acc:.1f}% accuracy predicting battle outcomes")
print(f"   2. {best_model_name} performed best among 3 models tested")
print(f"   3. Feature importance shows {rf_feature_importance.iloc[0]['feature']} is most critical")
print(f"   4. Model demonstrates deck composition has predictive power")

## Insights for Presentation

**Key Points**:
1. Achieved X% accuracy (compare to 56.94% benchmark)
2. Top 3 most important features are: [list]
3. This means: [actionable insight from feature importance]